In [51]:
from basic_rl_envi_for_dam_bidding_main.import_data_example import *

# Get Data

In [106]:
start = pd.Timestamp(year=2023, month=3, day=1, tz="utc")
#end = pd.Timestamp(year=2022, month=5, day=1, tz="utc")
end = pd.Timestamp.now(tz="utc")
# get the end of the current day
end = end.floor('D') + pd.Timedelta(days=2)

In [149]:
df_demand = get_demand(start, end)
df_vre = get_vre(start, end)
df_gen = get_gen(start, end)
df_mcp = get_mcp(start, end)
#states = get_states_list(start, end)

In [180]:
# get day ahead market price for each day
df_mcp_hourly = df_mcp.pivot_table(index=df_mcp.index.date, columns=df_mcp.index.hour, values=0)
df_mcp_hourly.index = pd.to_datetime(df_mcp_hourly.index)
df_mcp_hourly = df_mcp_hourly.add_prefix('market_price_hour_')
df_mcp_hourly['end_auction'] = (df_mcp_hourly.index - pd.Timedelta(hours = 12))
#df_mcp_hourly = df_mcp_hourly.reset_index(drop=False).set_index('end_auction')
#df_mcp_hourly = df_mcp_hourly.rename(columns={'index': 'day'})

In [181]:
# df_demand: 2023-05-08 23:45:00+02:00 gibt den Load für 23:45-00:00 Uhr an

In [189]:
# get forecasted load for each hour of each day.
df_demand_hourly = df_demand.pivot_table(index=df_demand.index.date, columns=df_demand.index.hour, values='Forecasted Load', aggfunc='sum')
df_demand_hourly = df_demand_hourly.add_prefix('forecasted_load_hour_')

In [187]:
df = pd.concat([df_demand_hourly, df_mcp_hourly], axis=1)

In [190]:
df_vre

,Solar,Wind Offshore,Wind Onshore
2023-03-01 01:00:00+01:00,0.0,247.0,6297.0
2023-03-01 01:15:00+01:00,0.0,244.0,6238.0
2023-03-01 01:30:00+01:00,0.0,243.0,6165.0
2023-03-01 01:45:00+01:00,0.0,241.0,6090.0
2023-03-01 02:00:00+01:00,0.0,239.0,6016.0
...,...,...,...
2023-05-08 22:45:00+02:00,0.0,2403.0,15929.0
2023-05-08 23:00:00+02:00,0.0,2401.0,16114.0
2023-05-08 23:15:00+02:00,0.0,2399.0,16148.0
2023-05-08 23:30:00+02:00,0.0,2397.0,16174.0


In [ ]:
# 2023-05-08 12:00:00+02:00,63621.0,29065.0,2153.0,10199.0,63511.0,80.41
# dieser Datenpunkt entspricht den Werten von 12:00 Uhr bis 12:30 Uhr
# Day ahead total load

In [196]:
# aggregate hourly
df_vre_agg = df_vre.groupby([pd.Grouper(freq='1h')])[['Solar', 'Wind Offshore', 'Wind Onshore']].sum()

In [205]:

test = df_vre_agg.pivot_table(index=df_vre_agg.index.date, columns=df_vre_agg.index.hour, values=['Wind Onshore', 'Wind Offshore', 'Solar'], aggfunc='sum')
#df_vre_hourly = df_vre_hourly.add_prefix('forecasted_load_hour_')

In [212]:
[test[col].add_prefix(f'{col}_hour_') for col in test.columns][0]

('Solar', 0)_hour_2023-03-01    NaN
('Solar', 0)_hour_2023-03-02    0.0
('Solar', 0)_hour_2023-03-03    0.0
('Solar', 0)_hour_2023-03-04    0.0
('Solar', 0)_hour_2023-03-05    0.0
                               ... 
('Solar', 0)_hour_2023-05-04    0.0
('Solar', 0)_hour_2023-05-05    0.0
('Solar', 0)_hour_2023-05-06    0.0
('Solar', 0)_hour_2023-05-07    0.0
('Solar', 0)_hour_2023-05-08    0.0
Name: (Solar, 0), Length: 69, dtype: float64

In [210]:
pd.concat([test['Solar'].add_prefix('Solar_hour_'), ])

,Solar_hour_0,Solar_hour_1,Solar_hour_2,Solar_hour_3,Solar_hour_4,Solar_hour_5,Solar_hour_6,Solar_hour_7,Solar_hour_8,Solar_hour_9,...,Solar_hour_14,Solar_hour_15,Solar_hour_16,Solar_hour_17,Solar_hour_18,Solar_hour_19,Solar_hour_20,Solar_hour_21,Solar_hour_22,Solar_hour_23
2023-03-01,NaN,0.0,0.0,0.0,0.0,0.0,22.0,5263.0,29740.0,61422.0,...,93677.0,66432.0,31765.0,5837.0,32.0,0.0,0.0,0.0,0.0,0.0
2023-03-02,0.0,0.0,0.0,0.0,0.0,0.0,36.0,5089.0,27789.0,56714.0,...,90641.0,64691.0,31098.0,5685.0,40.0,0.0,0.0,0.0,0.0,0.0
2023-03-03,0.0,0.0,0.0,0.0,0.0,0.0,24.0,3892.0,20536.0,42817.0,...,83441.0,60028.0,29677.0,5670.0,58.0,0.0,0.0,0.0,0.0,0.0
2023-03-04,0.0,0.0,0.0,0.0,0.0,0.0,32.0,3523.0,17251.0,33544.0,...,52068.0,35620.0,17389.0,3598.0,66.0,0.0,0.0,0.0,0.0,0.0
2023-03-05,0.0,0.0,0.0,0.0,0.0,0.0,36.0,2669.0,12839.0,25921.0,...,34399.0,24114.0,12628.0,2872.0,83.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-05-04,0.0,0.0,0.0,0.0,0.0,116.0,5980.0,27338.0,60536.0,95511.0,...,140625.0,124407.0,98780.0,65430.0,32631.0,10607.0,1538.0,63.0,0.0,0.0
2023-05-05,0.0,0.0,0.0,0.0,0.0,81.0,4947.0,23056.0,49723.0,76896.0,...,93326.0,77801.0,57524.0,37106.0,20015.0,7142.0,1073.0,53.0,0.0,0.0
2023-05-06,0.0,0.0,0.0,0.0,0.0,90.0,3340.0,15702.0,36115.0,58381.0,...,102036.0,90951.0,73419.0,52032.0,28546.0,9819.0,1302.0,51.0,0.0,0.0
2023-05-07,0.0,0.0,0.0,0.0,0.0,129.0,5159.0,20356.0,43788.0,69961.0,...,103188.0,89505.0,70400.0,48366.0,26624.0,9588.0,1342.0,31.0,0.0,0.0


In [64]:
# aggregate daily
df.groupby([pd.Grouper(freq='1d')])[['Forecasted Load', 'Solar', 'Wind Offshore', 'Wind Onshore', 'Actual Aggregated']].sum()

,Forecasted Load,Solar,Wind Offshore,Wind Onshore,Actual Aggregated
2022-01-01 00:00:00+01:00,4087580.0,130566.0,467309.0,1384558.0,1150624.0
2022-01-02 00:00:00+01:00,4893343.0,94954.0,556874.0,2463120.0,1497235.0
2022-01-03 00:00:00+01:00,5790309.0,102006.0,588415.0,3092386.0,1804336.0
2022-01-04 00:00:00+01:00,6026514.0,43926.0,366535.0,1336172.0,1530429.0
2022-01-05 00:00:00+01:00,6223270.0,110089.0,608893.0,2748791.0,1846391.0
...,...,...,...,...,...
2022-04-27 00:00:00+02:00,5501443.0,867811.0,24266.0,244662.0,1409259.0
2022-04-28 00:00:00+02:00,5452723.0,1067294.0,75181.0,364603.0,1461638.0
2022-04-29 00:00:00+02:00,5404809.0,1015177.0,63335.0,339967.0,1421279.0
2022-04-30 00:00:00+02:00,4727408.0,659786.0,38224.0,407541.0,1200306.0


In [ ]:
#

# Bedeutung der einzelnen Variablen
- Forecasted Load:
- Solar
- Wind Offshore:
- Wind Onshore:
- day ahead market price: EUR/MWh
